In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import yfinance as yf
import time
from datetime import date
import re

In [3]:
#Reading the table
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = tables[0]
#Create a DataFrame with company tickers, names, and the year they were added.
df_clean = df[['Symbol','Security','Date added']].copy()
df_clean.head()

,Symbol,Security,Date added
0,MMM,3M,1957-03-04
1,AOS,A. O. Smith,2017-07-26
2,ABT,Abbott Laboratories,1957-03-04
3,ABBV,AbbVie,2012-12-31
4,ACN,Accenture,2011-07-06


In [4]:
#1.Extract the year from the addition date and calculate the number of stocks added each year.
df_clean['Date added'] = pd.to_datetime(df['Date added'])
df_clean['Year'] = df_clean['Date added'].dt.year
grouped_stocks = df_clean.groupby('Year')['Symbol'].count().sort_values(ascending=False)
grouped_stocks
#2016 and 2017

,Symbol
Year,
1957,53
2016,23
2017,23
2019,22
2008,17
2024,16
2022,16
2023,15
2021,15


In [5]:
stocks = """United States - S&P 500 (^GSPC)
China - Shanghai Composite (000001.SS)
Hong Kong - HANG SENG INDEX (^HSI)
Australia - S&P/ASX 200 (^AXJO)
India - Nifty 50 (^NSEI)
Canada - S&P/TSX Composite (^GSPTSE)
Germany - DAX (^GDAXI)
United Kingdom - FTSE 100 (^FTSE)
Japan - Nikkei 225 (^N225)
Mexico - IPC Mexico (^MXX)
Brazil - Ibovespa (^BVSP)"""
tickers = re.findall(r'\(([^)]+)\)', stocks)
tickers

['^GSPC',
 '000001.SS',
 '^HSI',
 '^AXJO',
 '^NSEI',
 '^GSPTSE',
 '^GDAXI',
 '^FTSE',
 '^N225',
 '^MXX',
 '^BVSP']

In [6]:
#2.How many indexes (out of 10) have better year-to-date returns than the US (S&P 500) as of May 1, 2025?
start_date = '2025-01-01'
end_date = end_date='2025-05-01'
result = {}
for ticker in tickers:
  ticker_obj = yf.Ticker(ticker)
  daily = ticker_obj.history(start=start_date, end=end_date)
  ytd = (daily['Close'].iloc[-1]/daily['Close'].iloc[0])-1
  result[ticker_obj.info['symbol']] = ytd
result


{'^GSPC': np.float64(-0.05103300748245043),
 '000001.SS': np.float64(0.005048172440500043),
 '^HSI': np.float64(0.1272001783592147),
 '^AXJO': np.float64(-0.009145002952478443),
 '^NSEI': np.float64(0.02490423740978498),
 '^GSPTSE': np.float64(-0.002261257179291487),
 '^GDAXI': np.float64(0.12346378381499523),
 '^FTSE': np.float64(0.028425901908435414),
 '^N225': np.float64(-0.08297930804200304),
 '^MXX': np.float64(0.13049444457570325),
 '^BVSP': np.float64(0.12438709677419357)}

In [7]:
#Creating a df from the result dictionary to see it better
df_result = pd.DataFrame(result.items(),columns=['Ticker','YTD'])
df_result["YTD %"] = (df_result["YTD"]*100).round(2)
df_result.sort_values(by='YTD %',ascending=False).reset_index(drop=True)

,Ticker,YTD,YTD %
0,^MXX,0.130494,13.05
1,^HSI,0.127200,12.72
2,^BVSP,0.124387,12.44
3,^GDAXI,0.123464,12.35
4,^FTSE,0.028426,2.84
5,^NSEI,0.024904,2.49
6,000001.SS,0.005048,0.50
7,^GSPTSE,-0.002261,-0.23
8,^AXJO,-0.009145,-0.91
9,^GSPC,-0.051033,-5.10


In [8]:
#3.[Index] S&P 500 Market Corrections Analysis
ticker_obj = yf.Ticker("^SPX")
daily_SP500 = ticker_obj.history(start = "1950-01-01")
daily_SP500["All-time-high-points"] = daily_SP500['Close']>daily_SP500['Close'].cummax().shift(1)
daily_SP500

,Open,High,Low,Close,Volume,Dividends,Stock Splits,All-time-high-points
Date,,,,,,,,
1950-01-03 00:00:00-05:00,16.660000,16.660000,16.660000,16.660000,1260000,0.0,0.0,False
1950-01-04 00:00:00-05:00,16.850000,16.850000,16.850000,16.850000,1890000,0.0,0.0,True
1950-01-05 00:00:00-05:00,16.930000,16.930000,16.930000,16.930000,2550000,0.0,0.0,True
1950-01-06 00:00:00-05:00,16.980000,16.980000,16.980000,16.980000,2010000,0.0,0.0,True
1950-01-09 00:00:00-05:00,17.080000,17.080000,17.080000,17.080000,2520000,0.0,0.0,True
...,...,...,...,...,...,...,...,...
2025-05-23 00:00:00-04:00,5781.890137,5829.509766,5767.410156,5802.819824,4662820000,0.0,0.0,False
2025-05-27 00:00:00-04:00,5854.069824,5924.330078,5854.069824,5921.540039,5366380000,0.0,0.0,False
2025-05-28 00:00:00-04:00,5925.540039,5939.919922,5881.879883,5888.549805,4665050000,0.0,0.0,False


In [9]:
#Create empty columns for min values
daily_SP500["Min_Between_Highs"] = False
daily_SP500["Min_Between_Highs_Value"] = None
daily_SP500["Drawdown"] = None
daily_SP500["Duration_of_Drawdown"] = None

#Generate the All time high points df for looping
all_time_highs = daily_SP500[daily_SP500["All-time-high-points"]==True]

#Loop through the ATH df for finding min values and the drawdown
for i in range(len(all_time_highs)-1):
  start = all_time_highs.index[i]
  end = all_time_highs.index[i+1]

  period_data = daily_SP500.loc[start:end].iloc[1:]
  ath_value = daily_SP500.loc[start, "Close"]

  if not period_data.empty:
    min_date = period_data["Close"].idxmin()
    min_value = period_data.loc[min_date, "Close"]
    drawdown = ((ath_value - min_value) / ath_value)*100
    daily_SP500.at[min_date, "Min_Between_Highs"] = True
    daily_SP500.at[min_date, "Min_Between_Highs_Value"] = min_value
    daily_SP500.at[min_date, "Drawdown"] = drawdown
    if drawdown >=5:
      duration_drawdown = (min_date - start).days
      daily_SP500.at[min_date, "Duration_of_Drawdown"] = duration_drawdown

corrections_df = daily_SP500[daily_SP500["Drawdown"]>5].sort_values(by="Drawdown",ascending=False)




In [10]:
median_duration = corrections_df["Duration_of_Drawdown"].median()
print(f"Median duration: {median_duration:.0f} days")

Median duration: 39 days


In [12]:
#Determine the 25th, 50th (median), and 75th percentiles for correction durations
df_percentiles = corrections_df["Duration_of_Drawdown"].quantile([0.25,0.5,0.75])
df_percentiles

,Duration_of_Drawdown
0.25,21.5
0.50,39.0
0.75,89.0


In [23]:
#Earnings Surprise Analysis for Amazon (AMZN)
ticker_obj = yf.Ticker("AMZN")
daily_amzn = ticker_obj.history()
daily_amzn["two_days_diff"] = (daily_amzn["Close"] / daily_amzn["Close"].shift(2))-1
daily_amzn

,Open,High,Low,Close,Volume,Dividends,Stock Splits,two_days_diff
Date,,,,,,,,
2025-04-30 00:00:00-04:00,182.169998,185.050003,178.850006,184.419998,55176500,0.0,0.0,NaN
2025-05-01 00:00:00-04:00,190.630005,191.809998,187.500000,190.199997,74266000,0.0,0.0,NaN
2025-05-02 00:00:00-04:00,191.440002,192.880005,186.399994,189.979996,77903500,0.0,0.0,0.030149
2025-05-05 00:00:00-04:00,186.509995,188.179993,185.529999,186.350006,35217500,0.0,0.0,-0.020242
2025-05-06 00:00:00-04:00,184.570007,187.929993,183.850006,185.009995,29314100,0.0,0.0,-0.026161
2025-05-07 00:00:00-04:00,185.559998,190.990005,185.009995,188.710007,43948600,0.0,0.0,0.012664
2025-05-08 00:00:00-04:00,191.429993,194.330002,188.820007,192.080002,41043600,0.0,0.0,0.038214
2025-05-09 00:00:00-04:00,193.380005,194.690002,191.160004,193.059998,29663100,0.0,0.0,0.023051
2025-05-12 00:00:00-04:00,210.710007,211.660004,205.750000,208.639999,75205000,0.0,0.0,0.086214


In [21]:
#[Stocks] Earnings Surprise Analysis for Amazon (AMZN)
url = "https://raw.githubusercontent.com/DataTalksClub/stock-markets-analytics-zoomcamp/main/cohorts/2025/ha1_Amazon.csv"
df_amzn = pd.read_csv(url,delimiter=";")
df_amzn.head(20)

,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise (%)
0,AMZN,Amazon.com Inc,"April 29, 2026 at 6 AM EDT",-,-,-
1,AMZN,Amazon.com Inc,"February 4, 2026 at 4 PM EST",-,-,-
2,AMZN,Amazon.com Inc,"October 29, 2025 at 6 AM EDT",-,-,-
3,AMZN,Amazon.com Inc,"July 30, 2025 at 4 PM EDT",-,-,-
4,AMZN,"Amazon.com, Inc.","May 1, 2025 at 4 PM EDT",???.36,???.59,+16.74
5,AMZN,"Amazon.com, Inc.","February 6, 2025 at 4 PM EST",???.49,???.86,+24.47
6,AMZN,"Amazon.com, Inc.","October 31, 2024 at 4 PM EDT",???.14,???.43,+25.17
7,AMZN,"Amazon.com, Inc.","August 1, 2024 at 4 PM EDT",01.???,???.26,+22.58
8,AMZN,"Amazon.com, Inc.","April 30, 2024 at 4 PM EDT",0.83,0.98,+17.91
9,AMZN,"Amazon.com, Inc.","February 1, 2024 at 4 PM EST",0.8,1,+24.55


In [39]:
#Identify positive earnings surprises (where "actual EPS > estimated EPS" OR "Surprise (%)>0"

#Data Cleaning
df_amzn['EPS Estimate'] = (df_amzn['EPS Estimate'].astype(str)
    .str.replace(r'\?', '', regex=True)
    .replace('-', np.nan)
    .replace('', np.nan)
    .astype(float)
)
# Explicitly convert to string before using .str accessor
df_amzn['Reported EPS'] = (df_amzn['Reported EPS'].astype(str)
    .str.replace(r'\?', '', regex=True)
    .replace('-', np.nan)
    .replace('', np.nan)
    .astype(float)
)
# Explicitly convert to string before using .str accessor
df_amzn['Surprise (%)'] = (df_amzn['Surprise (%)'].astype(str)
    .replace({r'\+': '', '%': '', r'\?': '', '-': ''}, regex=True)
    .replace('', np.nan)
    .astype(float)
)
df_amzn


,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise (%)
0,AMZN,Amazon.com Inc,"April 29, 2026 at 6 AM EDT",NaN,NaN,NaN
1,AMZN,Amazon.com Inc,"February 4, 2026 at 4 PM EST",NaN,NaN,NaN
2,AMZN,Amazon.com Inc,"October 29, 2025 at 6 AM EDT",NaN,NaN,NaN
3,AMZN,Amazon.com Inc,"July 30, 2025 at 4 PM EDT",NaN,NaN,NaN
4,AMZN,"Amazon.com, Inc.","May 1, 2025 at 4 PM EDT",0.36,0.59,16.74
...,...,...,...,...,...,...
112,AMZN,"Amazon.com, Inc.","April 27, 1998 at 12 AM EDT",NaN,NaN,13.92
113,AMZN,"Amazon.com, Inc.","January 22, 1998 at 12 AM EST",NaN,NaN,11.41
114,AMZN,"Amazon.com, Inc.","October 27, 1997 at 12 AM EST",NaN,NaN,13.29
115,AMZN,"Amazon.com, Inc.","July 10, 1997 at 12 AM EDT",NaN,NaN,13.33


In [40]:
df_amzn['Positive_surprise'] = (df_amzn['Reported EPS'] > df_amzn['EPS Estimate']) | (df_amzn['Surprise (%)'] > 0)
df_amzn

,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise (%),Positive_surprise
0,AMZN,Amazon.com Inc,"April 29, 2026 at 6 AM EDT",NaN,NaN,NaN,False
1,AMZN,Amazon.com Inc,"February 4, 2026 at 4 PM EST",NaN,NaN,NaN,False
2,AMZN,Amazon.com Inc,"October 29, 2025 at 6 AM EDT",NaN,NaN,NaN,False
3,AMZN,Amazon.com Inc,"July 30, 2025 at 4 PM EDT",NaN,NaN,NaN,False
4,AMZN,"Amazon.com, Inc.","May 1, 2025 at 4 PM EDT",0.36,0.59,16.74,True
...,...,...,...,...,...,...,...
112,AMZN,"Amazon.com, Inc.","April 27, 1998 at 12 AM EDT",NaN,NaN,13.92,True
113,AMZN,"Amazon.com, Inc.","January 22, 1998 at 12 AM EST",NaN,NaN,11.41,True
114,AMZN,"Amazon.com, Inc.","October 27, 1997 at 12 AM EST",NaN,NaN,13.29,True
115,AMZN,"Amazon.com, Inc.","July 10, 1997 at 12 AM EDT",NaN,NaN,13.33,True


In [42]:
#Calculate 2-day percentage changes following positive earnings surprises. Show your answer in % (closest number to the 2nd digit): return * 100.0
df_amzn_positive = df_amzn[df_amzn['Positive_surprise'] == True]
df_amzn_positive['two_days_diff'] = (df_amzn_positive['Reported EPS'] / df_amzn_positive['Reported EPS'].shift(2))-1
df_amzn_positive

<ipython-input-42-734f5ff04d53>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_amzn_positive['two_days_diff'] = (df_amzn_positive['Reported EPS'] / df_amzn_positive['Reported EPS'].shift(2))-1


,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise (%),Positive_surprise,two_days_diff
4,AMZN,"Amazon.com, Inc.","May 1, 2025 at 4 PM EDT",0.36,0.59,16.74,True,NaN
5,AMZN,"Amazon.com, Inc.","February 6, 2025 at 4 PM EST",0.49,0.86,24.47,True,NaN
6,AMZN,"Amazon.com, Inc.","October 31, 2024 at 4 PM EDT",0.14,0.43,25.17,True,-0.271186
7,AMZN,"Amazon.com, Inc.","August 1, 2024 at 4 PM EDT",1.00,0.26,22.58,True,-0.697674
8,AMZN,"Amazon.com, Inc.","April 30, 2024 at 4 PM EDT",0.83,0.98,17.91,True,1.279070
...,...,...,...,...,...,...,...,...
111,AMZN,"Amazon.com, Inc.","July 22, 1998 at 12 AM EDT",NaN,NaN,1.34,True,NaN
112,AMZN,"Amazon.com, Inc.","April 27, 1998 at 12 AM EDT",NaN,NaN,13.92,True,NaN
113,AMZN,"Amazon.com, Inc.","January 22, 1998 at 12 AM EST",NaN,NaN,11.41,True,NaN
114,AMZN,"Amazon.com, Inc.","October 27, 1997 at 12 AM EST",NaN,NaN,13.29,True,NaN


In [45]:
returns = []

for date in df_amzn_positive['Earnings Date']:
    # Find the first trading day on or after the earnings date
    date_idx = daily_amzn[daily_amzn['Earnings Date'] >= date].index.min()

    try:
        # Use iloc to get positions safely
        row_pos = daily_amzn.index.get_loc(date_idx)
        price_day1 = daily_amzn.iloc[row_pos]['Close']
        price_day3 = daily_amzn.iloc[row_pos + 2]['Close']
        ret = (price_day3 / price_day1) - 1
    except (KeyError, IndexError):
        ret = np.nan  # Handle if day3 is out of bounds or missing

    returns.append(ret)

df_amzn_positive['2_day_return'] = returns


KeyError: 'Earnings Date'

In [46]:
# Ensure 'Earnings Date' in df_amzn_positive is a datetime object
df_amzn_positive['Earnings Date'] = pd.to_datetime(df_amzn_positive['Earnings Date'])

returns = []

for earnings_date in df_amzn_positive['Earnings Date']:
    # Find the first trading day on or after the earnings date using the index
    date_idx = daily_amzn.index[daily_amzn.index >= earnings_date].min()

    try:
        # Use get_loc to find the integer position of the trading date
        row_pos = daily_amzn.index.get_loc(date_idx)
        price_day1 = daily_amzn.iloc[row_pos]['Close']
        # Check if the index position for day3 exists
        if row_pos + 2 < len(daily_amzn.index):
            price_day3 = daily_amzn.iloc[row_pos + 2]['Close']
            ret = (price_day3 / price_day1) - 1
        else:
            ret = np.nan # Handle cases where day3 is out of bounds
    except (KeyError, IndexError):
        # This catch might be less likely now, but kept for robustness
        ret = np.nan
    except ValueError:
         # Handle cases where date_idx is NaT (no trading day found on or after earnings_date)
         ret = np.nan

    returns.append(ret)

df_amzn_positive['2_day_return'] = returns

<ipython-input-46-65488019946d>:2: FutureWarning: Parsed string "May 1, 2025 at 4 PM EDT" included an un-recognized timezone "EDT". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  df_amzn_positive['Earnings Date'] = pd.to_datetime(df_amzn_positive['Earnings Date'])
<ipython-input-46-65488019946d>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_amzn_positive['Earnings Date'] = pd.to_datetime(df_amzn_positive['Earnings Date'])
<ipython-input-46-65488019946d>:2: FutureWarning: Parsed string "February 6, 2025 at 4 PM EST" included an un-recognized timezone "EST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_loca

TypeError: Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp

In [49]:
# Ensure 'Earnings Date' in df_amzn_positive is a datetime object
df_amzn_positive['Earnings Date'] = pd.to_datetime(df_amzn_positive['Earnings Date'])

returns = []

# Get the timezone from the daily_amzn index
index_timezone = daily_amzn.index.tz

for earnings_date in df_amzn_positive['Earnings Date']:
    # Localize the earnings_date to the same timezone as daily_amzn.index
    # Handle NaT values in earnings_date before localization
    if pd.isna(earnings_date):
        returns.append(np.nan)
        continue

    earnings_date_localized = earnings_date.tz_localize(index_timezone)

    # Find the first trading day on or after the localized earnings date using the index
    # Use searchsorted for potentially better performance on large indices
    try:
        # searchsorted finds the index where earnings_date_localized would be inserted to maintain order
        # We want the first element *greater than or equal to*, so search for the first index
        # where the index is >= earnings_date_localized
        pos = daily_amzn.index.searchsorted(earnings_date_localized, side='left')

        # Handle cases where the earnings date is after the last trading day
        if pos >= len(daily_amzn.index):
            ret = np.nan
        else:
            date_idx = daily_amzn.index[pos] # This is the first trading day >= earnings_date_localized

            # Use get_loc to find the integer position of the trading date
            row_pos = daily_amzn.index.get_loc(date_idx)

            price_day1 = daily_amzn.iloc[row_pos]['Close']

            # Check if the index position for day3 exists
            if row_pos + 2 < len(daily_amzn.index):
                price_day3 = daily_amzn.iloc[row_pos + 2]['Close']
                ret = (price_day3 / price_day1) - 1
            else:
                ret = np.nan # Handle cases where day3 is out of bounds

    except Exception:
        # Catch potential errors during searchsorted or get_loc if dates are weird
        ret = np.nan


    returns.append(ret)

df_amzn_positive['2_day_return'] = returns
mean_2day_return_pct = df_amzn_positive['2_day_return'].mean() * 100.0
mean_2day_return_pct

<ipython-input-49-c63d7c509a03>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_amzn_positive['Earnings Date'] = pd.to_datetime(df_amzn_positive['Earnings Date'])
<ipython-input-49-c63d7c509a03>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_amzn_positive['2_day_return'] = returns


np.float64(2.964580010707488)